# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix,make_scorer
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [39]:
# define functions
def clean_data(data_train):
    data_train['story']=data_train['story'].str.replace(",","")
    data_train['story']=data_train['story'].str.replace(".","")
    return data_train

def vectorize_data(X_train,X_test):
    count_vect = CountVectorizer(ngram_range=(1,2),min_df=5, max_features=10000)
    X_train_counts = count_vect.fit_transform(X_train)
    X_test_counts = count_vect.transform(X_test)
    return X_train_counts, X_test_counts,count_vect


def scale_data(X_train_counts):
    scaler = MinMaxScaler()
    X_train_counts = scaler.fit_transform(X_train_counts.toarray())
    return X_train_counts


def train_model(model_name,model, X_train_counts, Y_train):
    if model_name == 'MultinomialNB':
        param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0,100.0], 'fit_prior': [False,True]}
        grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=10, scoring='f1_macro')
        grid_search.fit(X_train_counts, Y_train)
        best_model = grid_search.best_estimator_
    else:
        best_model = model
    best_model.fit(X_train_counts,Y_train)
    scoring_male = make_scorer(f1_score, pos_label='m')
    scoring_female = make_scorer(f1_score, pos_label='f')
    scoring_avg = 'f1_macro'
    f1_scorer=make_scorer(f1_score,average='macro')
    scores_male = cross_val_score(best_model, X_train_counts, Y_train, cv=5, scoring=scoring_male)
    scores_female = cross_val_score(best_model, X_train_counts, Y_train, cv=5, scoring=scoring_female)
    scores_avg = cross_val_score(best_model, X_train_counts, Y_train, cv=5, scoring=f1_scorer)
    
    male_f1 = scores_male.mean()
    female_f1 = scores_female.mean()
    avg_f1 = scores_avg.mean()
    
    return male_f1,female_f1,avg_f1


def main():
    # load data
    train_filename = 'annotated_corpus_for_train.csv'
    test_filename  = 'corpus_for_test.csv'
    df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
    df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')
    # clean data
    df_train = clean_data(df_train)

    # split data into training and test sets
    X_train, Y_train = df_train['story'], df_train['gender']
    X_test = df_test['story']

    # vectorize data
    X_train_counts,X_test_counts, count_vect = vectorize_data(X_train,X_test)

    # scale data
    X_train_counts_scaled = scale_data(X_train_counts)

    # define models
    models = {
        'MultinomialNB': MultinomialNB(),
        'Perceptron': Perceptron(alpha=0.01, max_iter=100),
        'LinearSVC': LinearSVC(),
        'KNeighborsClassifier': KNeighborsClassifier(),
        'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=10)
    }

    # train and evaluate models
    for model_name, model in models.items():
        print(f'********************** {model_name} **********************')
        
        # train and evaluate unscaled model
        print('Unscaled data:')
        male_f1, female_f1,avg_f1 = train_model(model_name,model, X_train_counts, Y_train)
        print(f"F1 score (male): {male_f1}")
        print(f"F1 score (female): {female_f1}")
        print(f"Average F1 score: {avg_f1}\n")

        # train and evaluate scaled model
        print('Scaled data:')
        X_train_counts_scaled = scale_data(X_train_counts)
        male_f1, female_f1,avg_f1 = train_model(model_name,model, X_train_counts_scaled, Y_train)
        print(f"F1 score (male): {male_f1}")
        print(f"F1 score (female): {female_f1}")
        print(f"Average F1 score: {avg_f1}\n")


if __name__ == '__main__':
    main()


********************** MultinomialNB **********************
Unscaled data:
F1 score (male): 0.8756227247532771
F1 score (female): 0.5303986560702978
Average F1 score: 0.7030106904117875

Scaled data:
F1 score (male): 0.8573075362585358
F1 score (female): 0.525336625815499
Average F1 score: 0.6913220810370175

********************** Perceptron **********************
Unscaled data:
F1 score (male): 0.8631502394627152
F1 score (female): 0.4389664291352474
Average F1 score: 0.6510583342989813

Scaled data:
F1 score (male): 0.864534165333739
F1 score (female): 0.38340644834942045
Average F1 score: 0.6239703068415798

********************** LinearSVC **********************
Unscaled data:
F1 score (male): 0.8636855978434943
F1 score (female): 0.5016294582610372
Average F1 score: 0.6858678582094333

Scaled data:
F1 score (male): 0.872318526018721
F1 score (female): 0.28801242236024843
Average F1 score: 0.5801654741894847

********************** KNeighborsClassifier **********************
Unsca

In [43]:
# Predicting the test type('m' / 'f') based to the trainning data from the 'MultinomialNB' model
X_train, Y_train = df_train['story'], df_train['gender']
X_test = df_test['story']
X_train_counts,X_test_counts, count_vect = vectorize_data(X_train,X_test)
model_name='MultinomialNB'
Multinomial_= MultinomialNB().fit(X_train_counts,Y_train)
male_f1, female_f1,avg_f1 = train_model(model_name,MultinomialNB(), X_train_counts, Y_train)

print(f"F1 score (male): {male_f1}")
print(f"F1 score (female): {female_f1}")
print(f"Average F1 score: {avg_f1}\n")

X_new_vect = count_vect.transform(df_test['story'])

y_new_pred = Multinomial_.predict(X_new_vect)

df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': y_new_pred})
df_predicted
unique_values, value_counts = np.unique(y_new_pred, return_counts=True)
for i in range(len(unique_values)):
    print(f"{unique_values[i]} count: {value_counts[i]}")

F1 score (male): 0.8780902559810281
F1 score (female): 0.5245338307247318
Average F1 score: 0.7013120433528799



,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


f count: 42
m count: 281


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [83]:
df_predicted.to_csv('classification_results.csv',index=False)